In [1]:
import numpy as np
import tensorflow as tf
import gzip
import pickle
import sys
import copy
import os.path
import pdb
import re
from run_split import SplitMnistGenerator, SplitMnistRandomGenerator
from run_permuted import PermutedMnistGenerator
from alg.cla_models_multihead import MFVI_NN, Vanilla_NN
from alg.HIBP_BNN_multihead import HIBP_BNN
from alg.IBP_BNN_multihead import IBP_BNN
from hibp_weight_pruning import prune_weights, MnistGenerator
from alg.utils import get_scores, get_scores_entropy, concatenate_results, get_uncertainties
from alg.vcl import run_vcl
from copy import deepcopy

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

/nfs/home/samuelk/anaconda2/envs/tf-gpu-cuda8/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/nfs/home/samuelk/anaconda2/envs/tf-gpu-cuda8/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/nfs/home/samuelk/anaconda2/envs/tf-gpu-cuda8/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/nfs/home/sam

# Split MNIST

In [2]:
hidden_size = [100]
batch_size = 128
no_epochs = 100
run = 2
val = False
cl2=True
cl3=False
#data_gen = SplitMnistRandomGenerator(val)
data_gen = SplitMnistGenerator(val, cl3)
all_acc, all_acc_ent = np.array([]), np.array([]) 
all_uncerts_split, accs = {}, {}
all_x_testsets, all_y_testsets = [], []
x_testsets, y_testsets = [], []

for task_id in range(data_gen.max_iter):
    np.random.seed(1)
    if val:
        _, _, x_test, y_test, _, _ = data_gen.next_task()
    else:
        _, _, x_test, y_test = data_gen.next_task()
    all_x_testsets.append(x_test)
    all_y_testsets.append(y_test)

data_gen.reset_cur_iter()
coreset_size = 0
single_head=False
in_dim, out_dim = data_gen.get_dims()
for task_id in range(data_gen.max_iter):

    tf.reset_default_graph()  
    tf.set_random_seed(1)
    np.random.seed(1)
    if val:
        x_train, y_train, x_test, y_test, _, _ = data_gen.next_task()
    else:
        x_train, y_train, x_test, y_test = data_gen.next_task()
    x_testsets.append(x_test)
    y_testsets.append(y_test)

    # Set the readout head to train
    head = 0 if single_head else task_id
    bsize = x_train.shape[0] if (batch_size is None) else batch_size

    # Train network with maximum likelihood to initialize first model
    if task_id == 0:
        ml_model = Vanilla_NN(in_dim, hidden_size, out_dim, x_train.shape[0])
        ml_model.train(x_train, y_train, task_id, 10, bsize)
        mf_weights = ml_model.get_weights()
        mf_variances = None
        mf_betas = None
        ml_model.close_session()

    # Train on non-coreset data
    model = IBP_BNN(input_size=in_dim, 
                     hidden_size=hidden_size, 
                     output_size=out_dim, 
                     training_size=x_train.shape[0], 
                     no_train_samples=10,
                     no_pred_samples=100,
                     num_ibp_samples=10, prev_means=mf_weights, 
                     prev_log_variances=mf_variances, 
                     prev_betas=mf_betas,
                     learning_rate=0.001, learning_rate_decay=0.87,
                     prior_mean=0.0, prior_var=0.7,
                     alpha0=5.0, beta0=1.0,
                     lambda_1=1.0, lambda_2=1.0,
                     tensorboard_dir='../ddm/logs',
                     #name='ibp_rs_opt_split_random_noise_rs_l1_mh_run{0}_task{1}'.format(run, task_id+1),
                     name='split_normal_run{0}_split_normal_l1_mh_task{1}'.format(run, task_id+1),
                     #name='ibp_perm_run{0}_perm_l1_mh_new_task{1}'.format(run, task_id+1),
                     use_local_reparam=False, implicit_beta=True)
    # graph_split_normal_run3_split_normal_l1_mh_task4
    # graph_ibp_rs_opt_split_random_noise_rs_l1_mh_run1_task1
    # graph_ibp_perm_run1_perm_l1_mh_new_task1
    model.create_model()
    #model.restore(os.path.join("logs_rs", 'graph_ibp_rs_opt_{0}'.format('split_random_noise_rs_l1_mh_run{0}_task{1}'.format(run, task_id+1))))
    model.restore(model.log_folder)
    
    mf_weights, mf_variances, mf_betas = model.get_weights()
    
    acc = get_scores(model, x_testsets, y_testsets, bsize, single_head)
    acc_ent = get_scores_entropy(model, x_testsets, y_testsets, bsize, data_gen.max_iter)
    all_acc = concatenate_results(acc, all_acc)
    all_acc_ent = concatenate_results(acc_ent, all_acc_ent)
    
    model.close_session()






Epoch: 0001 cost= 0.053171204
Epoch: 0006 cost= 0.000736305
Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Z: (1, ?, 100)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../ddm/logs/graph_split_normal_run2_split_normal_l1_mh_task1/model.ckpt
Z: (1, ?, 100)
INFO:tensorflow:Restoring parameters from ../ddm/logs/graph_split_normal_run2_split_normal_l1_mh_task2/model.ckpt
Z: (1, ?, 100)
INFO:tensorflow:Restoring parameters from ../ddm/logs/graph_split_normal_run2_split_normal_l1_mh_task3/model.ckpt
Z: (1, ?, 100)
INFO:tensorflow:Restoring parameters from ../ddm/logs/graph_split_normal_run2_split_normal_l1_mh_task4/model.ckpt
Z: (1, ?, 100)
INFO:tensorflow:Restoring parameters from ../ddm/logs/graph_split_normal_run2_split_normal_l

In [3]:
all_acc

array([[0.9984421 ,        nan,        nan,        nan,        nan],
       [0.98419577, 0.97999464,        nan,        nan,        nan],
       [0.8547196 , 0.95980069, 0.99275712,        nan,        nan],
       [0.73139583, 0.94299157, 0.98591057, 0.99436583,        nan],
       [0.75060867, 0.93856502, 0.97401715, 0.97940608, 0.98178935]])

In [6]:
np.nanmean(all_acc)

0.936597330611126

In [4]:
all_acc_ent

array([[0.99827706,        nan,        nan,        nan,        nan],
       [0.9835478 , 0.97983678,        nan,        nan,        nan],
       [0.84818763, 0.96132315, 0.99132965,        nan,        nan],
       [0.73607537, 0.94078869, 0.98609616, 0.99474668,        nan],
       [0.58356247, 0.55603075, 0.97380868, 0.9789752 , 0.98039559]])

In [5]:
np.nanmean(all_acc_ent)

0.8995321111379837

# Permuted MNIST

In [2]:
hidden_size = [100]
batch_size = 128
no_epochs = 100
run = 2
val = True
data_gen = PermutedMnistGenerator(max_iter=5, val=val)
all_acc, all_acc_ent = np.array([]), np.array([]) 
x_testsets, y_testsets = [], []

# permutations are need to be correct
for task_id in range(data_gen.max_iter):
    np.random.seed(1)
    if val:
        _, _, x_test, y_test, _, _ = data_gen.next_task()
    else:
        _, _, x_test, y_test = data_gen.next_task()

data_gen.reset_cur_iter()
coreset_size = 0
single_head=False
in_dim, out_dim = data_gen.get_dims()
for task_id in range(data_gen.max_iter):

    tf.reset_default_graph()  
    tf.set_random_seed(1)
    np.random.seed(1)
    if val:
        x_train, y_train, x_test, y_test, _, _ = data_gen.next_task()
    else:
        x_train, y_train, x_test, y_test = data_gen.next_task()
    x_testsets.append(x_test)
    y_testsets.append(y_test)

    # Set the readout head to train
    head = 0 if single_head else task_id
    bsize = x_train.shape[0] if (batch_size is None) else batch_size

    # Train network with maximum likelihood to initialize first model
    if task_id == 0:
        ml_model = Vanilla_NN(in_dim, hidden_size, out_dim, x_train.shape[0])
        ml_model.train(x_train, y_train, task_id, 10, bsize)
        mf_weights = ml_model.get_weights()
        mf_variances = None
        mf_betas = None
        ml_model.close_session()

    # Train on non-coreset data
    model = IBP_BNN(input_size=in_dim, 
                     hidden_size=hidden_size, 
                     output_size=out_dim, 
                     training_size=x_train.shape[0], 
                     no_train_samples=10,
                     no_pred_samples=100,
                     num_ibp_samples=10, prev_means=mf_weights, 
                     prev_log_variances=mf_variances, 
                     prev_betas=mf_betas,
                     learning_rate=0.001, learning_rate_decay=0.87,
                     prior_mean=0.0, prior_var=0.7,
                     alpha0=5.0, beta0=1.0,
                     lambda_1=1.0, lambda_2=1.0,
                     tensorboard_dir='../ddm/logs_perm',
                     #name='ibp_rs_opt_split_random_noise_rs_l1_mh_run{0}_task{1}'.format(run, task_id+1),
                     #name='split_normal_run{0}_split_normal_l1_mh_task{1}'.format(run, task_id+1),
                     name='ibp_perm_run{0}_perm_l1_mh_new_task{1}'.format(run, task_id+1),
                     use_local_reparam=False, implicit_beta=True)
    # graph_split_normal_run3_split_normal_l1_mh_task4
    # graph_ibp_rs_opt_split_random_noise_rs_l1_mh_run1_task1
    # graph_ibp_perm_run1_perm_l1_mh_new_task1
    model.create_model()
    #model.restore(os.path.join("logs_rs", 'graph_ibp_rs_opt_{0}'.format('split_random_noise_rs_l1_mh_run{0}_task{1}'.format(run, task_id+1))))
    model.restore(model.log_folder)
    
    mf_weights, mf_variances, mf_betas = model.get_weights()
    
    acc = get_scores(model, x_testsets, y_testsets, bsize, single_head)
    acc_ent = get_scores_entropy(model, x_testsets, y_testsets, bsize, data_gen.max_iter)
    all_acc = concatenate_results(acc, all_acc)
    all_acc_ent = concatenate_results(acc_ent, all_acc_ent)
    
    model.close_session()






Epoch: 0001 cost= 0.460927513
Epoch: 0006 cost= 0.087805614
Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Z: (1, ?, 100)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../ddm/logs_perm/graph_ibp_perm_run2_perm_l1_mh_new_task1/model.ckpt
> /nfs/home/samuelk/IBP_BNN/src/alg/utils.py(88)get_scores_entropy()
-> head = np.argmin(uncerts)
(Pdb) c
Z: (1, ?, 100)
INFO:tensorflow:Restoring parameters from ../ddm/logs_perm/graph_ibp_perm_run2_perm_l1_mh_new_task2/model.ckpt
> /nfs/home/samuelk/IBP_BNN/src/alg/utils.py(88)get_scores_entropy()
-> head = np.argmin(uncerts)
(Pdb) c
> /nfs/home/samuelk/IBP_BNN/src/alg/utils.py(87)get_scores_entropy()
-> pdb.set_trace()
(Pdb) uncerts
[0.828159, 0.2618167, 2.2624934, 2.263195, 2.263002]
(Pdb

In [3]:
all_acc

array([[0.9535977 ,        nan,        nan,        nan,        nan],
       [0.94893295, 0.93817939,        nan,        nan,        nan],
       [0.9403392 , 0.92611155, 0.94079213,        nan,        nan],
       [0.9393928 , 0.92944324, 0.93385779, 0.93991891,        nan],
       [0.93400712, 0.92507812, 0.91658129, 0.93453026, 0.94003758]])

In [4]:
all_acc_ent

array([[0.95370748,        nan,        nan,        nan,        nan],
       [0.94896855, 0.93819521,        nan,        nan,        nan],
       [0.94024229, 0.9255805 , 0.93989122,        nan,        nan],
       [0.93934336, 0.92972508, 0.93446301, 0.93967563,        nan],
       [0.93397449, 0.92504055, 0.91725573, 0.93454905, 0.94088014]])